In [1]:
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
import networkx as nx
from nltk.tokenize import  sent_tokenize

In [2]:
df = pd.read_csv('Data/tennis_articles_v4.csv')
df['article_text']

0    Maria Sharapova has basically no friends as te...
1    BASEL, Switzerland (AP), Roger Federer advance...
2    Roger Federer has revealed that organisers of ...
3    Kei Nishikori will try to end his long losing ...
4    Federer, 37, first broke through on tour over ...
5    Nadal has not played tennis since he was force...
6    Tennis giveth, and tennis taketh away. The end...
7    Federer won the Swiss Indoors last week by bea...
Name: article_text, dtype: object

In [3]:
import re

# Mengganti karakter non-alfabet dengan spasi
s = 'he&&&s'
s = re.sub("[^a-zA-Z]", " ", s)

In [4]:
# Pembersihan kalimat, menghapus karakter non-alfabet dan mengubah menjadi huruf kecil

dict = {}  # Dictionary untuk menyimpan kalimat asli
s = ""  # Inisialisasi string kosong

# Menggabungkan semua teks artikel menjadi satu string
for a in df['article_text']:
    s += a
s = s.lower()  # Mengubah seluruh teks menjadi huruf kecil

# Memecah teks menjadi kalimat-kalimat
sentences = sent_tokenize(s)

final = []  # Daftar untuk menyimpan kalimat yang sudah dibersihkan

# Membersihkan setiap kalimat dan menyimpannya dalam 'final' dan 'dict'
for s in sentences:
    temp = re.sub("[^a-zA-Z]", " ", s)  # Hapus karakter non-alfabet
    temp = temp.lower()  # Ubah kalimat menjadi huruf kecil
    final.append(temp)  # Tambahkan kalimat yang dibersihkan
    dict[temp] = s  # Simpan kalimat asli dengan kata yang dibersihkan sebagai key

In [5]:
# Fungsi untuk menghitung kemiripan antara dua kalimat menggunakan vektor kata
def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []  # Jika tidak ada stopwords, buat list kosong
    
    # Membuat daftar kata untuk kedua kalimat dan mengubahnya ke huruf kecil
    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]

    # Gabungkan semua kata unik dari kedua kalimat
    all_words = list(set(sent1 + sent2))

    # Membuat vektor kosong untuk kedua kalimat
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)

    # Isi vektor untuk kalimat pertama berdasarkan kata yang ada
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1

    # Isi vektor untuk kalimat kedua berdasarkan kata yang ada
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1

    # Mengembalikan jarak kosinus antara kedua kalimat
    return 1 - cosine_distance(vector1, vector2)

# Fungsi untuk membuat matriks kemiripan antar kalimat
def build_similarity_matrix(sentences, stop_words):
    # Membuat matriks kosong dengan ukuran len(sentences) x len(sentences)
    similarity_matrix = np.zeros((len(sentences), len(sentences)))

    # Mengisi matriks dengan kemiripan antar kalimat
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2:  # Lewati jika kalimat yang dibandingkan adalah kalimat yang sama
                continue 
            # Menghitung kemiripan kalimat idx1 dan idx2
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)
    
    return similarity_matrix  # Mengembalikan matriks kemiripan

In [6]:
# Menghasilkan ringkasan teks dengan menggunakan algoritma PageRank untuk menentukan kalimat-kalimat yang paling penting dalam sebuah dokumen

# Step 2 - Membuat Matriks Kemiripan Antar Kalimat
sentence_similarity_martix = build_similarity_matrix(final, '')

# Step 3 - Memberi Peringkat pada Kalimat
sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_martix)
scores = nx.pagerank(sentence_similarity_graph)

# Step 4 - Menyortir Kalimat Berdasarkan Peringkat
ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(final)), reverse=True)    

# Step 5 - Menghasilkan Ringkasan
# print('Summarize Text: \n', ". ".join(summarize_text))

In [7]:
for i in range(1):
     print(dict[ranked_sentence[i][1]])

argentina and britain received wild cards to the new-look event, and will compete along with the four 2018 semi-finalists and the 12 teams who win qualifying rounds next february.
